# Calculate biomes in B-SOSE 
# following criteria in Fay & McKinley 2014
# save resulting new biome file as netcdf

In [1]:
#get_ipython().system(u'jupyter nbconvert --to=python MASTER_biomes_in_models_FayMcKinley2014_forLavinia.ipynb')

Traceback (most recent call last):
  File "/home/shklvn09/miniconda3/envs/py3_eddy/bin/jupyter", line 11, in <module>
    sys.exit(main())
  File "/home/shklvn09/miniconda3/envs/py3_eddy/lib/python3.8/site-packages/jupyter_core/command.py", line 247, in main
    command = _jupyter_abspath(subcommand)
  File "/home/shklvn09/miniconda3/envs/py3_eddy/lib/python3.8/site-packages/jupyter_core/command.py", line 133, in _jupyter_abspath
    raise Exception(
Exception: Jupyter command `jupyter-nbconvert` not found.


In [1]:
### modules
import os
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from netCDF4 import Dataset
from netCDF4 import Dataset
#from mpl_toolkits.basemap import Basemap
from matplotlib import cm 
import cartopy.crs as ccrs

In [ ]:
#------
# settings....
#------

# save resulting biomes as netcdf?
save_netcdf = True
# saving_directory  = '/scratch/usr/shklvn09/SDIR/Fay_McKinley_biomes/' 
# netcdf_name       = 'FayMcKinley_regions_SOsubs_orca05.nc' 

# save plots to...
save_to_dir ='/scratch/usr/shklvn09/SDIR/Fay_McKinley_biomes/figures/'
if not os.path.exists(save_to_dir):
    print('Created '+save_to_dir)
    os.makedirs(save_to_dir)

#------
# define paths to data files
#------
# NOTE: here, I have only taken top-layer chl in B-SOSE (2.1m), consider taking more than that?
# not quite sure what "MLD" I am looking at here in B-SOSE output... to be clarified!
ff_mld = Dataset('/work/ollie/ncara/observations/BSOSE/bsose_i133_2013to2018_1deg_monthly_MLD_regular.nc')
ff_chl = Dataset('/work/ollie/ncara/observations/BSOSE/bsose_i133_2013to2018_1deg_monthly_Chl_srf_regular.nc')
ff_ice = Dataset('/work/ollie/ncara/observations/BSOSE/bsose_i133_2013to2018_1deg_monthly_SeaIceArea_regular.nc')
ff_sst = Dataset('/work/ollie/ncara/observations/BSOSE/bsose_i133_2013to2018_1deg_monthly_Theta_srf_regular.nc')
# output is in time x lat x lon, with "time" being in months!

year1,year2 = 2013,2019
time = np.arange(year1,year2)
print(time)
    
#-----
# comparison to mask in RECCAP
#-----
path_reccap_mask = '/work/ollie/ncara/RECCAPv2/reccap_regions_SOsubs.nc'

In [ ]:
Dataset?

In [ ]:
#------
# load data (monthly data from 2013-2018)
#------

mld = ff_mld.variables['BLGMLD'][:,:,:]
chl = np.squeeze(ff_chl.variables['BLGCHL'][:,:,:,:]) # cdo command left depth dimension in variable with size "1"
ice = ff_ice.variables['SIarea'][:,:,:]
sst = np.squeeze(ff_sst.variables['THETA'][:,:,:,:])

lat = ff_ice.variables['lat'][:]
lon = ff_ice.variables['lon'][:]

print mld.shape # months x lat x lon
print chl.shape
print ice.shape
print sst.shape
print lat.shape
print lon.shape

In [ ]:
#-----
# FUNCTIONS (for post-processing of data accoridng to Fay & McKinley criteria)
#-----

def annualmax(datam):
        """
        this function calculates the annual max for a 3d (t,y,x) variable.

        """
        import numpy as np
        nn = np.shape(datam)
        nt = nn[0]
        
        dimt = len(nn) #time dimension along which to average!
        if dimt ==3:
            nx = nn[2]
            ny = nn[1]
            var_annualmax = np.nan * np.zeros((int(np.floor(nt/12)), ny, nx))
            for t in range(0,int(np.floor(nt/12))):
                ind1 = t*12
                ind2 = ind1 + 12
                datamB = datam[ind1:ind2,:,:]
                var_annualmax[t,:,:] = np.nanmax(datamB, axis=0)
        else:
            raise ValueError('Input dimensions not suitable.')
        return var_annualmax
    

def annualmean(datam):
        """
        this function calculates the annual mean for a 3d (t,y,x) variable.

        """
        import numpy as np
        nn = np.shape(datam)
        nt = nn[0]
        
        dimt = len(nn) #time dimension along which to average!
        if dimt ==3:
            nx = nn[2]
            ny = nn[1]
            var_annualmean = np.nan * np.zeros((int(np.floor(nt/12)), ny, nx))
            for t in range(0,int(np.floor(nt/12))):
                ind1 = t*12
                ind2 = ind1 + 12
                #print ind2
                datamB = datam[ind1:ind2,:,:]
                var_annualmean[t,:,:] = np.nanmean(datamB, axis=0)
        else:
            raise ValueError('Input dimensions not suitable.')
        return var_annualmean
    
    
def djfm_mean(datam):
        """
        this function calculates the DJFM max for a 3d (t,y,x) variable. 
        Assumes that the months go from January-December, so DJFM corresponds
        to the indices [12,1,2,3]

        """
        import numpy as np
        nn = np.shape(datam)
        nt = nn[0]
        
        dimt = len(nn) #time dimension along which to average!
        if dimt ==3:
            nx = nn[2]
            ny = nn[1]
            var_djfm_mean = np.nan * np.zeros((int(np.floor(nt/12)), ny, nx))
            for t in range(0,int(np.floor(nt/12))):
                ind1 = t*12
                ind2 = t*12+1
                ind3 = t*12+2
                ind4 = ind1 + 11
                datamB = datam[[ind1,ind2,ind3,ind4],:,:]
                var_djfm_mean[t,:,:] = np.nanmax(datamB, axis=0)
                
        else:
            raise ValueError('Input dimensions not suitable.')
        return var_djfm_mean

In [ ]:
#------
# POST-PROCESSING according to Fay & McKinley criteria
#------

# chl: at each grid cell, get DJFM mean for each year
chl_djfm = djfm_mean(chl)
#print chl_djfm.shape

# MLD: at each grid cell, get max MLD for each year
MLD_max = annualmax(mld)
#print MLD_max.shape

# SST: at each grid cell, get annual mean for each year
sst_annual = annualmean(sst)
#print sst_annual.shape

# ice area: at each grid cell, get annual mean for each year
#ice_annual = np.zeros([len(time),len(lat),len(lon)])
# Note (10.6.20): adapt to take annual max (area with at least 50% coverage at any time of the year)
ice_annual = annualmax(ice)
#ice_annual = annualmean(ice)
#print ice_annual.shape 

In [ ]:
#------
# calculate climatological mean for all variables
#------

chl_djfm_clim   = np.mean(chl_djfm,axis=0)
MLD_max_clim    = np.mean(MLD_max,axis=0)
sst_annual_clim = np.mean(sst_annual,axis=0)
ice_annual_clim = np.mean(ice_annual,axis=0)

# set fill values to NaN 
MLD_max_clim[MLD_max_clim>10000000.]=np.nan
chl_djfm_clim[chl_djfm_clim>10000000.]=np.nan
ice_annual_clim[ice_annual_clim>10000000.]=np.nan


In [ ]:
#-----
# FUNCTIONS (for plotting)
#-----

def subroutine_plotting_SO(data,lon,lat,levels,colormap,contour_line,\
                           label_colorbar,plot_title,save_to_dir,filename):
    # plot mean or std of co2 fluxes
    # provide flux in mol C m-2 yr-1

  #  ff = Dataset('/work/ollie/ncara/RECCAPv2/reccap_regions_SOsubs.nc')
  #  regions = ff.variables['SO_basins_biomes'][:,:]
  #  regions[np.isnan(regions)]=9 # w/o this line, the contouring of the regions is incomplete
    
    # repeat first element at the end to get rid of grey line in plot
    b = np.zeros((180,360+1))
    b[:,:-1] = data
    b[:,-1] = data[:,0]
    data = b
    del b
    
    lon=np.concatenate((lon,np.full(1,lon[0]))) 
    
    fig=plt.figure(figsize=(12,7), facecolor='w', edgecolor='k') 
    ax = fig.add_axes([0, 0, 1, 1])
    #map = Basemap(projection='robin',lon_0=0)
    map = Basemap(projection='spstere',boundinglat=-30,lon_0=-180,resolution='l')
    X, Y = np.meshgrid(lon, lat) 
    x, y = map(X,Y)
    map.drawmapboundary(fill_color='0.8')
    map.drawcoastlines()
    #mlabels=[False,False,False,False]
    #plabels=[True,True,True,True]		    
    #map.drawparallels(np.arange(-89.5,89.5,30),labels=plabels) #[1,0,0,0]
    #map.drawmeridians(np.arange(map.lonmin,map.lonmax+30,60),labels=mlabels) #[0,0,0,1]
    map.drawcoastlines(linewidth=0.25)
    #plt.annotate(plot_title,fontsize=25,fontweight='bold')
    plt.contourf(x, y, data, levels=levels, cmap=colormap, extend='both')#,labelsize=16)
    cbar=plt.colorbar(orientation='horizontal',shrink=0.5,pad=0.1)
    cbar.ax.set_xlabel(label_colorbar,fontsize=18)
    #plt.tick_params(labelsize=16)#
    #cbar.ax.set_xticklabels(fontsize=16)
    plt.contour(x, y, data, [contour_line],colors='k',lw=1.0)#,color='k') #, cmap=colormap, extend='both')
  #  plt.contour(x,y,regions,[0,1,2,3,4,5,6,7,8],colors='k',lw=1.0) #[0,1,2,3,4,5,6,7,8],
    plt.text(0.5,-0.09, plot_title, fontsize=20,fontweight='bold',\
         horizontalalignment='center',verticalalignment='center', transform=ax.transAxes)
    plt.savefig(save_to_dir+filename, bbox_inches='tight',dpi=150, transparent=False)     
    #plt.savefig(save_to_dir+modelname+'_'+region+'_std_dev_annual_mean_co2_flux_'+str(year1)+'-'+str(year2)+\
    #            '_eval_stow_mismatch_aw.eps',bbox_inches='tight', format='eps',dpi=300, transparent=False)
    plt.show()
    
    
def subroutine_plotting_SO_biomes(data,lon,lat,levels,colormap,ctick_names,\
                           label_colorbar,plot_title,save_to_dir,filename):
    # plot mean or std of co2 fluxes
    # provide flux in mol C m-2 yr-1

  #  ff = Dataset('/work/ollie/ncara/RECCAPv2/reccap_regions_SOsubs.nc')
  #  regions = ff.variables['SO_basins_biomes'][:,:]
  #  regions[np.isnan(regions)]=9 # w/o this line, the contouring of the regions is incomplete
    
    # repeat first element at the end to get rid of grey line in plot
    b = np.zeros((180,360+1))
    b[:,:-1] = data
    b[:,-1] = data[:,0]
    data = b
    del b
    
    lon=np.concatenate((lon,np.full(1,lon[0]))) 
    
    fig=plt.figure(figsize=(12,7), facecolor='w', edgecolor='k') 
    ax = fig.add_axes([0, 0, 1, 1])
    map = Basemap(projection='spstere',boundinglat=-30,lon_0=-180,resolution='l')
    X, Y = np.meshgrid(lon, lat) 
    x, y = map(X,Y)
    map.drawmapboundary(fill_color='0.8')
    map.drawcoastlines()
    #mlabels=[False,False,False,False]
    #plabels=[True,True,True,True]		    
    #map.drawparallels(np.arange(-89.5,89.5,30),labels=plabels) #[1,0,0,0]
    #map.drawmeridians(np.arange(map.lonmin,map.lonmax+30,60),labels=mlabels) #[0,0,0,1]
    map.drawcoastlines(linewidth=0.25)
    #plt.annotate(plot_title,fontsize=25,fontweight='bold')
    plt.contourf(x, y, data, levels=levels, cmap=colormap)#, extend='both')
    cbar=plt.colorbar(orientation='horizontal',shrink=0.5,pad=0.1,ticks=[0.5,1.5,2.5])
    cbar.ax.set_xticklabels(ctick_names,fontsize=16) #rotation=45 
    cbar.ax.set_xlabel(label_colorbar,fontsize=18)
    #  plt.contour(x,y,regions,[0,1,2,3,4,5,6,7,8],colors='k',lw=1.0) #[0,1,2,3,4,5,6,7,8],
    plt.text(0.5,-0.09, plot_title, fontsize=20,fontweight='bold',\
         horizontalalignment='center',verticalalignment='center', transform=ax.transAxes)
    plt.savefig(save_to_dir+filename, bbox_inches='tight',dpi=150, transparent=False)     
    #plt.savefig(save_to_dir+modelname+'_'+region+'_std_dev_annual_mean_co2_flux_'+str(year1)+'-'+str(year2)+\
    #            '_eval_stow_mismatch_aw.eps',bbox_inches='tight', format='eps',dpi=300, transparent=False)
    plt.show()
    

In [ ]:
#-----
# plot fields underlying the biome definition
#-----

# 
levels1 = np.arange(0.0,1.1,0.1)   # sea ice cover
levels2 = np.arange(0.,320.,20.)  # max MLD
levels3 = np.arange(-2.,25.,1.) # SST
levels4 = np.arange(0.,1.1,0.1) # chl
use_color  = cm.Spectral_r  
#use_color2 = cm.RdBu_r  

# sea ice cover
label_colorbar = 'relative cover'
plot_title = 'B-SOSE annual max sea ice cover'
contour_line = 0.5
filename = 'BSOSE_clim_seaice_cover_annualmax.png'
subroutine_plotting_SO(ice_annual_clim,lon,lat,\
                    levels1,use_color,contour_line,label_colorbar,plot_title,save_to_dir,filename)

# SST
label_colorbar = '$^{\circ}$C'
plot_title = 'B-SOSE annual mean SST'
contour_line = 8.0
filename = 'BSOSE_clim_SST.png'
subroutine_plotting_SO(sst_annual_clim,lon,lat,\
                    levels3,use_color,contour_line,label_colorbar,plot_title,save_to_dir,filename)

# MLD 
label_colorbar = 'm'
plot_title = 'B-SOSE annual max MLD'
contour_line = 150
filename = 'BSOSE_clim_maxMLD.png'
subroutine_plotting_SO(MLD_max_clim,lon,lat,\
                    levels2,use_color,contour_line,label_colorbar,plot_title,save_to_dir,filename)

# chl
label_colorbar = 'mg chl m$^{-3}$'
plot_title = 'B-SOSE DJFM mean Chl'
contour_line = 0.16
filename = 'BSOSE_clim_srf_chl.png'
subroutine_plotting_SO(chl_djfm_clim,lon,lat,\
                    levels4,use_color,contour_line,label_colorbar,plot_title,save_to_dir,filename)

In [ ]:
#-----    
# determine biomes (circumpolar)
#-----

subregions = np.zeros([len(lat),len(lon)])
for i in range(0,len(lat)):
    for j in range(0,len(lon)):
        if ice_annual_clim[i,j]>0.5:  # ICE
            subregions[i,j] = 1
        elif ice_annual_clim[i,j]<=0.5 and sst_annual_clim[i,j]<8.0:  # SPSS
            subregions[i,j] = 2
        elif sst_annual_clim[i,j]>8.0 and (MLD_max_clim[i,j]>150 or chl_djfm_clim[i,j]>0.16): # STSS
            subregions[i,j] = 3

subregions[np.isnan(MLD_max_clim)]=np.nan
subregions[subregions==0]=np.nan

#-----
# compare to regions in RECCAP
#-----

ff = Dataset(path_reccap_mask)
all_regions = ff.variables['SO_basins_biomes'][:,:]
lon2 = ff.variables['lon'][:]
#print all_regions.shape
subregions_fay = np.zeros([180,360])
subregions_fay[all_regions==2]=1  # ICE
subregions_fay[all_regions==5]=1
subregions_fay[all_regions==8]=1
subregions_fay[all_regions==1]=2  # SPSS
subregions_fay[all_regions==4]=2
subregions_fay[all_regions==7]=2
subregions_fay[all_regions==0]=3  # STSS
subregions_fay[all_regions==3]=3
subregions_fay[all_regions==6]=3
subregions_fay[subregions_fay==0]=np.nan

#-----
# plot circumpolar subregions
#-----

levels5 = np.arange(0.,4.0,1.0) # biomes
ctick_names = ['ICE','SPSS','STSS']
label_colorbar = ''
plot_title = 'Biomes in B-SOSE'
filename = 'BSOSE_clim_biomes_ice_annualmax.png'
subroutine_plotting_SO_biomes(subregions,lon,lat,\
                    levels5,use_color,ctick_names,label_colorbar,plot_title,save_to_dir,filename)
plot_title = 'Biomes in Fay & McKinley'
filename = 'FayMcKinley_clim_biomes.png'
subroutine_plotting_SO_biomes(subregions_fay,lon2,lat,\
                    levels5,use_color,ctick_names,label_colorbar,plot_title,save_to_dir,filename)

In [ ]:
#-----
# save as netcdf
#-----
    
if save_netcdf: 
    # add a statement to only do the below if file does not yet exist.
    if not os.path.exists(saving_directory+netcdf_name):
        print 'Create file '+saving_directory+netcdf_name
        w_nc_fid = Dataset(saving_directory+netcdf_name, 'w', format='NETCDF4_CLASSIC')
        w_nc_fid.info= 'B-SOSE, 1/6°, based on climatological SST, ice cover, MLD and chl over 2013-2018'
        # create dimension & variable
        w_nc_fid.createDimension('lon', len(lon)) 
        w_nc_fid.createDimension('lat', len(lat)) 
        w_nc_var1 = w_nc_fid.createVariable('SO_basins_biomes', 'f8',('lat','lon'))
        w_nc_var2 = w_nc_fid.createVariable('lon', 'f8',('lon'))
        w_nc_var3 = w_nc_fid.createVariable('lat', 'f8',('lat'))         
        w_nc_var4 = w_nc_fid.createVariable('maxMLD', 'f8',('lat','lon'))
        w_nc_var5 = w_nc_fid.createVariable('chl', 'f8',('lat','lon'))
        w_nc_var6 = w_nc_fid.createVariable('SST', 'f8',('lat','lon'))
        w_nc_var7 = w_nc_fid.createVariable('ice', 'f8',('lat','lon'))
        w_nc_var1.description = 'subregions defined after criteria in Fay & McKinley 2014'
        w_nc_var4.long_name = 'max annual MLD'
        w_nc_var5.long_name = 'DJFM mean chlorophyll concentration'
        w_nc_var6.long_name = 'annual mean SST'
        w_nc_var7.long_name = 'annual max sea ice cover'
        w_nc_var4.units = 'm'
        w_nc_var5.units = 'mg chl m-3'
        w_nc_var6.units = 'deg C'
        w_nc_var7.units = 'n.d.'
        w_nc_fid.close()
    else:
        print 'File '+saving_directory+netcdf_name+' exists already, overwrite'
    
save_netcdf = False
if save_netcdf: 
    w_nc_fid = Dataset(saving_directory+netcdf_name, 'r+', format='NETCDF4_CLASSIC')      # Create and open new netcdf file to write to
    w_nc_fid.variables['SO_basins_biomes'][:,:] = subregions_model 
    w_nc_fid.variables['maxMLD'][:,:] = MLD_max_clim2 
    w_nc_fid.variables['chl'][:,:] = chl_djfm_clim2 
    w_nc_fid.variables['SST'][:,:] = sst_annual_clim2 
    w_nc_fid.variables['ice'][:,:] = ice_annual_clim2 
    w_nc_fid.variables['lon'][:] = lon2 # same as in Fay & McKinley
    w_nc_fid.variables['lat'][:] = lat
    w_nc_fid.close()  